In [2]:
import sys
import pandas as pd
import matplotlib.pyplot as plt

df_500030 = pd.read_csv('./data/500030_2024-07-01_2024-09-30.csv')
df_500030['patient_id'] = 500030
df_500030['date_original'] = df_500030['date']

df_679372 = pd.read_csv('./data/679372_2024-07-01_2024-09-30.csv')
df_679372['patient_id'] = 679372
df_679372['date_original'] = df_679372['date']

df = pd.concat([df_500030, df_679372])
df['interval'] = df['date'].apply(lambda x: x[27:29]) # trimming the head
df['date'] = df['date'].apply(lambda x: x[:19]) # trimming the tail
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')

C:\Users\paulp\AppData\Local\Temp\ipykernel_21456\3930358227.py:9: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_679372 = pd.read_csv('./data/679372_2024-07-01_2024-09-30.csv')


Start code Here

# Objective

Create Difference Plot for BGL users

In [9]:
def separate_date(df):
    df.loc[:, 'date'] = pd.to_datetime(df['date'], format='%y/%m/%d %H:%M:%S.%f')
    df.loc[:, 'year'] = df['date'].dt.year
    df.loc[:, 'month'] = df['date'].dt.month
    df.loc[:, 'day'] = df['date'].dt.day
    df.loc[:, 'hour'] = df['date'].dt.hour
    df.loc[:, 'minute'] = df['date'].dt.minute
    df.loc[:, 'second'] = df['date'].dt.second
    df.loc[:, 'seconds_with_micro'] = df['date'].dt.second + df['date'].dt.microsecond / 1_000_000
    return df



In [18]:
df_500030 = df[df['patient_id']==500030]
df_679372 = df[df['patient_id']==679372]

In [19]:
# Applying the function to the dataframe
separate_date(df_500030)

C:\Users\paulp\AppData\Local\Temp\ipykernel_21456\1411879342.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'], format='%y/%m/%d %H:%M:%S.%f')
C:\Users\paulp\AppData\Local\Temp\ipykernel_21456\1411879342.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['date'].dt.year
C:\Users\paulp\AppData\Local\Temp\ipykernel_21456\1411879342.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

,Unnamed: 0,date,sender_id,bgl,bgl_date_millis,text,template,msg_type,affects_fob,affects_iob,...,patient_id,date_original,interval,year,month,day,hour,minute,second,seconds_with_micro
0,0,2024-07-01 00:02:39,NaN,98.0,NaN,NaN,NaN,NaN,NaN,NaN,...,500030,2024-07-01 00:02:39-05:00,,2024,7,1,0,2,39,39.0
1,1,2024-07-01 00:07:39,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,...,500030,2024-07-01 00:07:39-05:00,,2024,7,1,0,7,39,39.0
2,2,2024-07-01 00:12:39,NaN,98.0,NaN,NaN,NaN,NaN,NaN,NaN,...,500030,2024-07-01 00:12:39-05:00,,2024,7,1,0,12,39,39.0
3,3,2024-07-01 00:17:39,NaN,94.0,NaN,NaN,NaN,NaN,NaN,NaN,...,500030,2024-07-01 00:17:39-05:00,,2024,7,1,0,17,39,39.0
4,4,2024-07-01 00:22:40,NaN,94.0,NaN,NaN,NaN,NaN,NaN,NaN,...,500030,2024-07-01 00:22:40-05:00,,2024,7,1,0,22,40,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39622,39533,2024-09-30 23:47:37,NaN,88.0,NaN,NaN,NaN,NaN,NaN,NaN,...,500030,2024-09-30 23:47:37-05:00,,2024,9,30,23,47,37,37.0
39623,39534,2024-09-30 23:49:38,NaN,90.0,NaN,NaN,NaN,NaN,NaN,NaN,...,500030,2024-09-30 23:49:38-05:00,,2024,9,30,23,49,38,38.0
39624,39535,2024-09-30 23:53:37,NaN,89.0,NaN,NaN,NaN,NaN,NaN,NaN,...,500030,2024-09-30 23:53:37-05:00,,2024,9,30,23,53,37,37.0
39625,39536,2024-09-30 23:55:38,NaN,88.0,NaN,NaN,NaN,NaN,NaN,NaN,...,500030,2024-09-30 23:55:38-05:00,,2024,9,30,23,55,38,38.0
